In [67]:
import numpy as np

import sys
sys.path.append("..")
from nc_suite import *

In [68]:
# Create img and labels
from torchvision import datasets
import cv2

img_baby = cv2.imread("../data/test/3.jpg",0)
img_baby = cv2.resize(img_baby, (28,28))

mnist = datasets.MNIST('data', train=True, download=True)
img = np.asarray(mnist[2][0])

label = np.copy(img) # shallow copy is fine, since it isn't python objects inside
label[label>0] = 1

In [69]:
# Really simple weighting methods...
# based on nc-test-multi these are the only consistent ones :)
# see TODO in nc-test-multi

# Image 1 (MNIST)
i_W = intensity_weight_matrix(img)
pi_W = intens_posit_wm(img)

# Image 2 (baby)
i_W_baby = intensity_weight_matrix(img_baby)
pi_W_baby = intens_posit_wm(img_baby)

In [70]:
from collections import OrderedDict

weights = OrderedDict(
    i_W=i_W, pi_W=pi_W,
    i_W_baby=i_W_baby, pi_W_baby=pi_W_baby
)

for key,value in weights.items(): #eigsh expects float or double
    weights[key] = value.astype(float)

In [72]:
def deterministic_vector_sign_flip(u):
    # from https://github.com/scikit-learn/scikit-learn/blob/main/sklearn/utils/extmath.py#L1097
    max_abs_rows = np.argmax(np.abs(u))
    signs = np.sign(u[max_abs_rows])
    u *= signs
    return u

In [ ]:
from functools import partial

# the methods to store
import scipy.linalg as linalg
import scipy.sparse.linalg as sparse

def laplace_expensive(D, W): # should be more expensive to compute...
    sqrt_D_inv = np.diag(1.0 / np.sqrt(np.diag(D))) # assumes D is 1 dimensional vector
    D = np.diag(D)
    return sqrt_D_inv @ (D - W) @ sqrt_D_inv

def laplace_cheap(D,W): # shift invert (implemented here) should be better...
    shift = 1
    sqrt_D = np.diag(np.sqrt(D)) # assumes D is 1 dimensional vector
    D = np.diag(D)
    return sqrt_D @ np.linalg.inv(D @ (1-shift) - W) @ sqrt_D

def post_solve_1(y, num=28):
    return y * num

def post_solve_2(y, D):
    # y = D^0.5 * z
    y = np.sqrt(D) @ y # e.g. if solving generalized need to do this to get y out of z...?
    return y

# TODO: Finish these
# Compute a difference in gradient between two inputs (one perturbed like gradSlam)
    

laplace_options = OrderedDict(
    # non_symm = non_symm,
    symm2 = laplace_expensive
)

eigs_options = OrderedDict(
    eigh_cff = partial(linalg.eigh, check_finite=False),
    lobpcg = partial(sparse.lobpcg, largest=False)
)

In [73]:
from tqdm import tqdm
    
# solve with any given eigensolver and laplacian approach
def generic_solve(W, laplace,solver):
    L = laplace(W)
    _, eig_vectors = solver(L)
    
    output = eig_vectors[:, 1]
    deterministic_vector_sign_flip(output)
    output = output.reshape(*np.sqrt(W.shape).astype(int)) 
    return output

def generic_solve_vals(W, laplace,solver):
    L = laplace(np.copy(W))
    vals, _ = solver(L)
    return vals

outputs = []
errors = []

row_headers = []
col_headers = []
titles = []

for laplace_name, laplace_func in laplace_options.items(): # and each laplacian type
    if laplace_name not in titles:
        titles.append(laplace_name)
    
    sub_output = []
    for eig_name, eig_func in eigs_options.items(): # for each eigensolver    
        row_headers.append(eig_name)
        
        for weight_name, weight in (pbar := tqdm(weights.items())): # and each weight    
            pbar.set_description(f'{eig_name} - {laplace_name}')
            
            if weight_name not in ['img','img_baby', 'label', 'blank']: # that isn't some other array      
                if weight_name not in col_headers:
                    col_headers.append(weight_name)
                try:
                    out1 = generic_solve(weight, laplace_func, eig_func)
                    sub_output.append(out1) # solve and append its output
                except Exception as err:
                    sub_output.append(np.zeros_like(img))
                    errors.append(f'{eig_name} - {weight_name}\n{err=}\n')
    outputs.append(sub_output)

eigh_cff - symm2: 100%|██████████| 8/8 [00:03<00:00,  2.44it/s]
